You are given an m x n matrix board containing letters 'X' and 'O', capture regions that are surrounded:

Connect: A cell is connected to adjacent cells horizontally or vertically.
Region: To form a region connect every 'O' cell.
Surround: The region is surrounded with 'X' cells if you can connect the region with 'X' cells and none of the region cells are on the edge of the board.
To capture a surrounded region, replace all 'O's with 'X's in-place within the original board. You do not need to return anything.

 

Example 1:

Input: board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]

Output: [["X","X","X","X"],["X","X","X","X"],["X","X","X","X"],["X","O","X","X"]]

Explanation:


In the above diagram, the bottom region is not captured because it is on the edge of the board and cannot be surrounded.

Example 2:

Input: board = [["X"]]

Output: [["X"]]

 

Constraints:

m == board.length
n == board[i].length
1 <= m, n <= 200
board[i][j] is 'X' or 'O'.

In [ ]:
# start from the border and build inside, mark all the 0 cells connected to border 0 as safe.

class Solution:
    def solve(self, board: list[list[str]]) -> None:
        if not board or not board[0]:
            return 
        
        n, m = len(board), len(board[0])
        q = []

        def bfs(i, j):
            q.append((i, j))
            board[i][j] = "S"  # Mark safe
            while q:
                # NOTE: no need to for loop over the q_len here... we can do continuously.
                x, y = q.pop(0)
                for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < n and 0 <= ny < m and board[nx][ny] == "O":
                        board[nx][ny] = "S"
                        q.append((nx, ny))

        # Step 1: Traverse boundary, mark connected 'O's as safe
        for i in range(n):
            # first row.
            if board[i][0] == "O":
                bfs(i, 0)

            # last row.
            if board[i][m-1] == "O":
                bfs(i, m-1)
        for j in range(m):
            # first col
            if board[0][j] == "O":
                bfs(0, j)

            # last col.
            if board[n-1][j] == "O":
                bfs(n-1, j)

        # Step 2: Flip surrounded 'O' → 'X', and safe 'S' → 'O'
        for i in range(n):
            for j in range(m):
                if board[i][j] == "O":
                    board[i][j] = "X"
                elif board[i][j] == "S":
                    board[i][j] = "O"

        print(board)

# tc - O(n * m)
# sc - O(n * m)

In [ ]:
# union find way.
# - Union border 'O's with safe region.
# - Union connected 'O's (so all the connected 0s will be in one group)
# - Flip not-safe 'O's, if the 0 cell is not connected with the safe region then mark it as 'X'.

class DSU:
    def __init__(self, n):
        self.parent = list(range(n))
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        # since we are gonna union with the safe region, always the seoncd one would be the parent.
        self.parent[px] = py


class Solution:
    def solve(self, board: list[list[str]]) -> None:
        if not board:
            return
        rows, cols = len(board), len(board[0])
        dsu = DSU(rows * cols + 1) # one extra cell for safe region.
        safe_region = rows * cols  # extra node/cell for border

        def index(r, c):
            return r * cols + c

        # Step 1: Union border 'O's with safe region.
        for r in range(rows):
            for c in range(cols):
                if board[r][c] == "O":
                    if r in [0, rows-1] or c in [0, cols-1]:
                        dsu.union(index(r, c), safe_region)

        # Step 2: Union connected 'O's
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        for r in range(rows):
            for c in range(cols):
                if board[r][c] == "O":
                    for dr, dc in directions:
                        nr, nc = r+dr, c+dc
                        if 0 <= nr < rows and 0 <= nc < cols and board[nr][nc] == "O":
                            dsu.union(index(r, c), index(nr, nc))

        # Step 3: Flip unconnected 'O's
        for r in range(rows):
            for c in range(cols):
                if board[r][c] == "O":
                    if dsu.find(index(r, c)) != dsu.find(safe_region):
                        board[r][c] = "X"


# Union-Find operations → nearly O(1) (amortized).
# We touch every cell once → O(mn) time.
# sc - O(N*M)

# both have the same time complexity:

- 🔹 Which is Optimal?

For competitive programming / interviews → DFS or BFS is usually expected (simpler).

For very large grids (e.g., thousands × thousands) → DSU avoids recursion overflow and is safer.

Both have same theoretical time complexity O(mn).

# So the optimal choice depends:

If interviewer says “solve it simply” → go with DFS/BFS.

If they hint at graph connectivity / union-find → use DSU.